# Part VII — Data, Integrations, and Advanced ORM  
## 33. Search, Files, and External Services (APIs, Webhooks, Storage, Reliability)

This chapter is about integrating Django with the outside world **safely** and
**reliably**, the way real production systems do:

- calling external APIs (timeouts, retries, error handling, idempotency)
- receiving webhooks (signatures, replay protection, fast acknowledgements)
- storing and serving files at scale (S3-like object storage, private media)
- keeping the architecture clean (service layer + tasks + audit logs + logs/metrics)

You’ll implement patterns that prevent the most common integration disasters:
- “requests hung and blocked all workers”
- “webhook retried 50 times and created duplicate data”
- “someone forged webhook requests”
- “uploaded files vanished after deploy”
- “API calls were flaky and caused cascading failures”

---

## 33.0 Learning Outcomes

By the end you should be able to:

1. Build an external API client module with:
   - timeouts (mandatory)
   - retries with backoff (only for retryable failures)
   - consistent error mapping (domain exceptions)
   - logging with request IDs
2. Use safe integration architecture:
   - views are thin
   - services encapsulate business rules
   - Celery handles slow/fragile work
3. Receive webhooks securely:
   - CSRF exempt (correctly)
   - signature verification (HMAC)
   - timestamp + replay protection
   - idempotency via unique event IDs
   - fast 2xx response + async processing
4. Store and serve files beyond local disk:
   - correct Django settings (`MEDIA_ROOT`, `STORAGES`)
   - object storage (S3-like) integration patterns
   - private media strategies (authenticated access / signed URLs)
5. Write tests for:
   - API client behavior (timeouts/retries) without real network
   - webhook signature verification and deduplication
   - file upload/download handling
6. Produce an integration checklist you can reuse in any Django app.

---

# 33.1 External APIs: “Call the network like a professional”

External calls fail. Not “maybe.” They will fail due to:
- timeouts
- DNS issues
- provider outages
- rate limits (429)
- transient 5xx
- partial failures (one endpoint slow, another fine)
- serialization issues

Your code must assume this and stay robust.

---

## 33.1.1 Non‑negotiables for external HTTP calls

### 1) Always set timeouts
Never do:

```python
requests.get(url)  # dangerous: can hang indefinitely
```

Always do:

```python
requests.get(url, timeout=5)
```

Because:
- without timeouts, a few stuck requests can exhaust your worker pool
- your entire web app can degrade and become unavailable

### 2) Retry only when retrying makes sense
Retry:
- network timeouts
- connection errors
- transient 502/503/504
- rate limiting with `Retry-After` (maybe, carefully)

Do NOT retry:
- 400/401/403 (client/auth mistakes)
- validation failures
- “not found” if that’s a real outcome

### 3) Use idempotency on create/charge endpoints
If you POST to create an order/payment and retry due to timeout, you might create
duplicates. You need idempotency keys (provider-specific pattern) or your own DB
dedupe keys.

### 4) Don’t block user requests on fragile integrations
For:
- sending emails
- calling slow third-party APIs
- syncing with external systems  
Prefer background tasks (Celery) and show “processing” status.

---

# 33.2 Build a Reusable HTTP Client Module (Sync)

We’ll implement a clean client using `requests` (sync). This is the most common
pattern for Django services and Celery tasks.

## 33.2.1 Add a small `core/http_client.py` (requests Session + retries)

Create `core/http_client.py`:

```python
from __future__ import annotations

import logging
from dataclasses import dataclass
from typing import Any, Optional

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

logger = logging.getLogger(__name__)


class ExternalApiError(Exception):
    """Base error for external API failures."""


class ExternalApiTimeout(ExternalApiError):
    pass


class ExternalApiUnavailable(ExternalApiError):
    pass


class ExternalApiBadResponse(ExternalApiError):
    pass


@dataclass(frozen=True)
class HttpResult:
    status_code: int
    json: Optional[dict[str, Any]]
    text: str


def build_retrying_session() -> requests.Session:
    """
    Returns a requests.Session configured with safe retry behavior.

    Retries are for:
    - connection errors
    - some transient 5xx
    - optionally 429 (rate limit), depending on your policy

    IMPORTANT: Retrying POST can be dangerous unless endpoint is idempotent.
    We keep retries to GET/HEAD/OPTIONS by default.
    """
    session = requests.Session()

    retry = Retry(
        total=3,
        connect=3,
        read=3,
        status=3,
        backoff_factor=0.5,
        status_forcelist=(429, 502, 503, 504),
        allowed_methods=frozenset(["GET", "HEAD", "OPTIONS"]),
        raise_on_status=False,
    )

    adapter = HTTPAdapter(max_retries=retry, pool_connections=10, pool_maxsize=10)
    session.mount("http://", adapter)
    session.mount("https://", adapter)

    return session


class JsonApiClient:
    def __init__(
        self,
        *,
        base_url: str,
        api_key: str | None = None,
        timeout_seconds: float = 5.0,
    ) -> None:
        self.base_url = base_url.rstrip("/")
        self.api_key = api_key
        self.timeout_seconds = timeout_seconds
        self.session = build_retrying_session()

    def _headers(self) -> dict[str, str]:
        headers = {"Accept": "application/json"}
        if self.api_key:
            headers["Authorization"] = f"Bearer {self.api_key}"
        return headers

    def get_json(self, path: str, params: dict[str, Any] | None = None) -> HttpResult:
        url = f"{self.base_url}/{path.lstrip('/')}"
        try:
            r = self.session.get(
                url,
                headers=self._headers(),
                params=params or {},
                timeout=self.timeout_seconds,
            )
        except requests.Timeout as e:
            raise ExternalApiTimeout(f"Timeout calling {url}") from e
        except requests.RequestException as e:
            raise ExternalApiUnavailable(f"Network error calling {url}") from e

        content_type = (r.headers.get("Content-Type") or "").lower()
        data = None
        if "application/json" in content_type:
            try:
                data = r.json()
            except ValueError:
                data = None

        if r.status_code >= 500:
            raise ExternalApiUnavailable(
                f"Upstream 5xx from {url} status={r.status_code}"
            )

        if r.status_code >= 400:
            raise ExternalApiBadResponse(
                f"Upstream 4xx from {url} status={r.status_code} body={r.text[:500]}"
            )

        return HttpResult(status_code=r.status_code, json=data, text=r.text)
```

### Explain the design (why it’s “industry standard”)

- **Session** reuses connections (faster and less resource-heavy than creating a new
  TCP connection per request).
- **Retry** is restricted to safe methods (GET/HEAD/OPTIONS) because retrying POST
  can create duplicates.
- We define **domain-specific exceptions**:
  - timeouts vs unavailable vs bad response
  so the rest of the app can decide:
  - retry in Celery
  - show a fallback UI
  - return 503
- We parse JSON only when `Content-Type` indicates JSON.

> If you later need retryable POST safely, you must use idempotency keys and a
> server-side “dedupe record” (we’ll cover patterns below).

---

# 33.3 External API Service Layer (Don’t call HTTP from views)

Create an `integrations` app to keep third-party logic separated from domain apps.

```bash
python manage.py startapp integrations
```

Add to `INSTALLED_APPS`:

```python
INSTALLED_APPS += ["integrations"]
```

## 33.3.1 Example: a “Rates” integration (toy, but realistic patterns)

Create `integrations/rates.py`:

```python
from __future__ import annotations

from dataclasses import dataclass
from typing import Any

from django.conf import settings

from core.http_client import JsonApiClient


@dataclass(frozen=True)
class ExchangeRate:
    base: str
    quote: str
    rate: float


def get_rates_client() -> JsonApiClient:
    return JsonApiClient(
        base_url=settings.RATES_API_BASE_URL,
        api_key=getattr(settings, "RATES_API_KEY", None),
        timeout_seconds=3.0,
    )


def fetch_exchange_rate(*, base: str, quote: str) -> ExchangeRate:
    client = get_rates_client()
    result = client.get_json("/rate", params={"base": base, "quote": quote})

    payload: dict[str, Any] = result.json or {}
    rate_raw = payload.get("rate")
    if rate_raw is None:
        raise ValueError("Upstream response missing 'rate'")

    return ExchangeRate(base=base, quote=quote, rate=float(rate_raw))
```

Add settings in `config/settings.py`:

```python
RATES_API_BASE_URL = "https://api.example.com"
RATES_API_KEY = ""  # set in env for production
```

### Why this separation matters
- `core.http_client` handles HTTP concerns (timeouts/retries/error mapping)
- `integrations.rates` handles domain mapping (ExchangeRate dataclass)
- views call `fetch_exchange_rate` and handle the result cleanly
- Celery tasks can reuse the same integration code

---

# 33.4 Reliability Pattern: “Fail fast, degrade gracefully”

When upstream is down:
- do you return 500?
- do you return cached data?
- do you queue work and respond “processing”?

Professional APIs/sites often do:
- quick 503 with a friendly message (for user-triggered real-time dependencies)
- or serve cached/stale data for read paths (e.g., “rates are 30 minutes old”)

## 33.4.1 Add caching around external calls (optional but common)

```python
from django.core.cache import cache

def fetch_exchange_rate_cached(*, base: str, quote: str) -> ExchangeRate:
    key = f"rate:{base}:{quote}"
    cached = cache.get(key)
    if cached:
        return ExchangeRate(**cached)

    rate = fetch_exchange_rate(base=base, quote=quote)
    cache.set(
        key,
        {"base": rate.base, "quote": rate.quote, "rate": rate.rate},
        timeout=60,
    )
    return rate
```

### Why caching helps
- reduces upstream usage
- mitigates transient upstream outages
- improves latency

But it introduces staleness; document TTL and UX.

---

# 33.5 Webhooks (Inbound Integrations): Correct Security + Idempotency

Webhooks are the inverse of external API calls:
- **they call you**
- they retry on failure
- they can be forged if you don’t verify signatures
- they must be processed quickly (respond 2xx fast)

---

## 33.5.1 The webhook “golden rules”
1. **Do not require CSRF** (webhooks aren’t browser forms).
2. **Do verify authenticity** (signature with secret).
3. **Do implement replay protection** (timestamp + idempotency).
4. **Respond quickly** (2xx) and process in background (Celery).
5. **Store raw payload** (or enough to debug) without leaking secrets.
6. **Never trust incoming data**; validate and map to your domain carefully.

---

## 33.6 Build a Webhook Receiver (HMAC Signature Verification)

We’ll create a generic webhook endpoint that expects:

- headers:
  - `X-Webhook-Id`: unique event id (string)
  - `X-Webhook-Timestamp`: unix timestamp
  - `X-Webhook-Signature`: HMAC SHA256 signature of `timestamp.payload`
- body:
  - JSON payload (bytes)

### 33.6.1 Create a model to store webhook events (idempotency + debugging)

In `integrations/models.py`:

```python
from __future__ import annotations

from django.db import models
from django.utils import timezone


class WebhookEvent(models.Model):
    class Status(models.TextChoices):
        RECEIVED = "received", "Received"
        PROCESSED = "processed", "Processed"
        FAILED = "failed", "Failed"

    provider = models.CharField(max_length=50)
    event_id = models.CharField(max_length=120)
    received_at = models.DateTimeField(default=timezone.now, editable=False)

    payload = models.JSONField()
    headers = models.JSONField(default=dict, blank=True)

    status = models.CharField(
        max_length=20,
        choices=Status.choices,
        default=Status.RECEIVED,
    )
    error = models.TextField(blank=True)

    processed_at = models.DateTimeField(null=True, blank=True)

    class Meta:
        constraints = [
            models.UniqueConstraint(
                fields=["provider", "event_id"],
                name="unique_provider_event_id",
            )
        ]
        indexes = [
            models.Index(fields=["provider", "-received_at"]),
            models.Index(fields=["provider", "status", "-received_at"]),
        ]

    def __str__(self) -> str:
        return f"{self.provider}:{self.event_id} ({self.status})"
```

Migrate:

```bash
python manage.py makemigrations
python manage.py migrate
```

### Why this model is critical
- Unique constraint prevents processing the same webhook twice (idempotency).
- Storing payload helps debugging and replaying (careful with sensitive data).
- Status fields support ops dashboards and retries.

---

## 33.6.2 Implement signature verification utility

Create `integrations/webhooks/security.py`:

```python
from __future__ import annotations

import hashlib
import hmac
import time


class WebhookSignatureError(Exception):
    pass


def compute_signature(*, secret: str, timestamp: str, body: bytes) -> str:
    """
    Compute hex HMAC-SHA256 of: f"{timestamp}.{body_as_bytes}"
    """
    msg = timestamp.encode("utf-8") + b"." + body
    mac = hmac.new(secret.encode("utf-8"), msg, hashlib.sha256)
    return mac.hexdigest()


def verify_signature(
    *,
    secret: str,
    timestamp: str,
    signature: str,
    body: bytes,
    tolerance_seconds: int = 300,
) -> None:
    # 1) Validate timestamp is recent to prevent replay.
    try:
        ts_int = int(timestamp)
    except ValueError as e:
        raise WebhookSignatureError("Invalid timestamp") from e

    now = int(time.time())
    if abs(now - ts_int) > tolerance_seconds:
        raise WebhookSignatureError("Timestamp outside tolerance window")

    # 2) Compute signature and compare in constant time.
    expected = compute_signature(secret=secret, timestamp=timestamp, body=body)

    if not hmac.compare_digest(expected, signature):
        raise WebhookSignatureError("Invalid signature")
```

### Why constant-time compare matters
`hmac.compare_digest` prevents timing attacks where attackers infer signatures from
response time differences.

---

## 33.6.3 Implement webhook view (fast ACK + Celery processing)

Create `integrations/views_webhooks.py`:

```python
from __future__ import annotations

import json
import logging

from django.conf import settings
from django.db import IntegrityError, transaction
from django.http import JsonResponse
from django.utils import timezone
from django.views.decorators.csrf import csrf_exempt
from django.views.decorators.http import require_POST

from integrations.models import WebhookEvent
from integrations.webhooks.security import WebhookSignatureError, verify_signature
from integrations.webhooks.tasks import process_webhook_event_task

logger = logging.getLogger(__name__)


@csrf_exempt
@require_POST
def provider_webhook(request):
    """
    Generic webhook receiver:
    - verifies signature
    - stores event with unique constraint
    - enqueues processing after commit
    - returns 200 quickly

    IMPORTANT: csrf_exempt is correct here because webhook calls are not browser
    submissions. Authenticity is enforced by signature verification.
    """
    provider = "provider_x"

    event_id = request.headers.get("X-Webhook-Id", "")
    timestamp = request.headers.get("X-Webhook-Timestamp", "")
    signature = request.headers.get("X-Webhook-Signature", "")

    if not event_id or not timestamp or not signature:
        return JsonResponse(
            {
                "error": {
                    "code": "missing_headers",
                    "message": "Missing webhook headers.",
                }
            },
            status=400,
        )

    body = request.body  # raw bytes
    try:
        verify_signature(
            secret=settings.WEBHOOK_PROVIDER_X_SECRET,
            timestamp=timestamp,
            signature=signature,
            body=body,
        )
    except WebhookSignatureError as e:
        logger.warning("Webhook signature error provider=%s event_id=%s", provider, event_id)
        return JsonResponse(
            {"error": {"code": "invalid_signature", "message": str(e)}},
            status=400,
        )

    try:
        payload = json.loads(body.decode("utf-8"))
    except (UnicodeDecodeError, json.JSONDecodeError):
        return JsonResponse(
            {"error": {"code": "invalid_json", "message": "Body must be JSON."}},
            status=400,
        )

    headers_to_store = {
        "X-Webhook-Id": event_id,
        "X-Webhook-Timestamp": timestamp,
        "X-Webhook-Signature": signature,
        "User-Agent": request.headers.get("User-Agent", ""),
    }

    try:
        with transaction.atomic():
            event = WebhookEvent.objects.create(
                provider=provider,
                event_id=event_id,
                payload=payload,
                headers=headers_to_store,
                status=WebhookEvent.Status.RECEIVED,
            )

            transaction.on_commit(
                lambda: process_webhook_event_task.delay(event.id)
            )
    except IntegrityError:
        # Duplicate event_id: treat as already received. Return 200 so upstream stops retrying.
        return JsonResponse({"status": "duplicate_ignored"}, status=200)

    return JsonResponse({"status": "accepted"}, status=200)
```

Add settings:

```python
WEBHOOK_PROVIDER_X_SECRET = "dev-secret-change-me"  # env var in production
```

Wire URL in `integrations/urls.py`:

```python
from django.urls import path
from integrations import views_webhooks

app_name = "integrations"

urlpatterns = [
    path("webhooks/provider-x/", views_webhooks.provider_webhook, name="provider_x_webhook"),
]
```

Include in `config/urls.py`:

```python
path("", include("integrations.urls")),
```

### Why return 200 on duplicates
Most providers treat any non-2xx as “retry later.” If you already processed the
event, you want retries to stop. Returning 200 is correct.

---

## 33.6.4 Process webhook asynchronously (Celery task)

Create `integrations/webhooks/tasks.py`:

```python
from __future__ import annotations

import logging

from celery import shared_task
from django.utils import timezone

from integrations.models import WebhookEvent

logger = logging.getLogger(__name__)


@shared_task(bind=True, autoretry_for=(Exception,), retry_backoff=True, retry_jitter=True, retry_kwargs={"max_retries": 5})
def process_webhook_event_task(self, event_pk: int) -> None:
    event = WebhookEvent.objects.get(pk=event_pk)

    if event.status == WebhookEvent.Status.PROCESSED:
        return

    try:
        payload = event.payload
        event_type = payload.get("type")

        # Example mapping: route based on payload type
        if event_type == "task.status_changed":
            # call domain service (recommended)
            _handle_task_status_changed(payload)
        elif event_type == "export.done":
            _handle_export_done(payload)
        else:
            logger.info("Unhandled webhook event type=%s id=%s", event_type, event.id)

        event.status = WebhookEvent.Status.PROCESSED
        event.processed_at = timezone.now()
        event.error = ""
        event.save(update_fields=["status", "processed_at", "error"])
    except Exception as e:
        event.status = WebhookEvent.Status.FAILED
        event.processed_at = timezone.now()
        event.error = str(e)
        event.save(update_fields=["status", "processed_at", "error"])
        raise


def _handle_task_status_changed(payload: dict) -> None:
    # Example placeholder for your real business logic:
    # - validate required keys
    # - lookup Task within correct org scope
    # - apply safe transition rules
    pass


def _handle_export_done(payload: dict) -> None:
    pass
```

### Why webhook processing must be idempotent too
Providers retry. Celery retries. Your code might run twice.
Therefore:
- use unique constraints
- check “already processed”
- design handlers to be safe on duplicates (e.g., “set status to done” is idempotent)

---

# 33.7 Webhook Testing (Signatures + Dedup)

## 33.7.1 Test signature verification utility

Create `integrations/tests/test_webhooks_security.py`:

```python
from django.test import TestCase

from integrations.webhooks.security import compute_signature, verify_signature, WebhookSignatureError


class WebhookSecurityTests(TestCase):
    def test_signature_round_trip(self):
        secret = "s"
        timestamp = "1700000000"
        body = b'{"type":"x"}'

        sig = compute_signature(secret=secret, timestamp=timestamp, body=body)
        # Should not raise
        verify_signature(secret=secret, timestamp=timestamp, signature=sig, body=body, tolerance_seconds=10**9)

    def test_bad_signature(self):
        with self.assertRaises(WebhookSignatureError):
            verify_signature(
                secret="s",
                timestamp="1700000000",
                signature="bad",
                body=b"{}",
                tolerance_seconds=10**9,
            )
```

## 33.7.2 Test receiver deduplication
Use Django test client to POST with headers and body.

```python
import time
from django.conf import settings
from django.test import TestCase
from django.urls import reverse

from integrations.models import WebhookEvent
from integrations.webhooks.security import compute_signature


class WebhookReceiverTests(TestCase):
    def _post_event(self, event_id: str):
        timestamp = str(int(time.time()))
        body = b'{"type":"task.status_changed","task_id":1}'
        sig = compute_signature(
            secret=settings.WEBHOOK_PROVIDER_X_SECRET,
            timestamp=timestamp,
            body=body,
        )

        return self.client.post(
            "/webhooks/provider-x/",
            data=body,
            content_type="application/json",
            **{
                "HTTP_X_WEBHOOK_ID": event_id,
                "HTTP_X_WEBHOOK_TIMESTAMP": timestamp,
                "HTTP_X_WEBHOOK_SIGNATURE": sig,
            },
        )

    def test_deduplicates_event_id(self):
        r1 = self._post_event("evt-1")
        self.assertEqual(r1.status_code, 200)
        self.assertEqual(WebhookEvent.objects.count(), 1)

        r2 = self._post_event("evt-1")
        self.assertEqual(r2.status_code, 200)
        self.assertEqual(WebhookEvent.objects.count(), 1)
```

---

# 33.8 File Storage Beyond Local Disk (S3-like Object Storage)

You already learned `MEDIA_ROOT`/`MEDIA_URL` for dev. In production, local disk is
often not enough because:
- multiple web servers don’t share the same filesystem
- deploys can replace instances and delete files
- backups become harder

So you move media to object storage:
- AWS S3
- MinIO
- DigitalOcean Spaces
- Google Cloud Storage (similar concept)

The standard Django approach is to use a **storage backend**.

---

## 33.8.1 Modern Django storage configuration (`STORAGES`)
In modern Django versions, you can configure storages like:

```python
STORAGES = {
    "default": {
        "BACKEND": "some.storage.Backend",
        "OPTIONS": {...},
    },
    "staticfiles": {
        "BACKEND": "django.contrib.staticfiles.storage.StaticFilesStorage",
    },
}
```

### Why this is better than old `DEFAULT_FILE_STORAGE`
- clearer separation between default storage (media) and staticfiles storage
- easier to configure multiple storages

> If your project is on older Django, you might see `DEFAULT_FILE_STORAGE`. Use
> `STORAGES` where available.

---

## 33.8.2 Common package: `django-storages` (S3 backend)
In many Django codebases, you’ll see:
- `django-storages` + boto3 (for S3-compatible storage)

Because provider configuration changes often, the “industry standard” here is the
**pattern**, not memorizing exact settings. The pattern:

1. Store credentials in env vars
2. Configure storage backend
3. Ensure `MEDIA_URL` points to your bucket/CDN domain
4. Consider public vs private media

### Public vs private media decision
- Public: cover images, public avatars
- Private: invoices, identity docs, internal exports

**You must decide per file type.** Often you use:
- one public bucket/prefix
- one private bucket/prefix with signed URLs

---

# 33.9 Private Media (Do NOT expose everything under /media/)

If a file must be private, you cannot just serve it from a public bucket or
`/media/` URL.

Two standard strategies:

## Strategy A — Signed URLs (object storage)
- user requests download
- server checks permissions
- server returns a short-lived signed URL to storage
- browser downloads directly from storage

Pros:
- scalable
- storage serves the bytes

Cons:
- requires signing integration with your storage provider

## Strategy B — Authenticated Django view streams the file
- user requests `/download/<id>/`
- Django checks permissions
- Django streams file using `FileResponse`

Pros:
- simple to implement
- works on local storage too

Cons:
- Django serves bytes (less scalable)
- needs careful caching headers

### Example: Authenticated export download (you already did)
Your `task_export_download` view is Strategy B. That’s a correct starter pattern.

---

# 33.10 Secure Webhook + Background + Realtime = Professional Integration Stack

You now have the professional triangle:

- Webhook receiver:
  - verifies signature
  - stores event (idempotency)
  - enqueues background processing
- Background worker:
  - updates domain state
  - logs/audits
  - broadcasts realtime notification if needed
- WebSocket:
  - pushes updates to connected clients (best effort)

This is a very common real-world architecture.

---

# 33.11 Integration Checklist (Use This Every Time)

## Outbound API calls
- [ ] timeout set on every request
- [ ] retries only for retryable errors
- [ ] POST endpoints idempotent before retrying
- [ ] errors mapped to domain exceptions
- [ ] logs include request_id and endpoint (not secrets)
- [ ] calls done in Celery if slow/unreliable

## Webhooks inbound
- [ ] CSRF exempt (correct)
- [ ] signature verified with constant-time compare
- [ ] timestamp tolerance check (replay mitigation)
- [ ] idempotency enforced via unique event id
- [ ] respond 2xx fast; async processing
- [ ] payload validated; unsafe inputs rejected
- [ ] audit/log events; store enough to debug

## Files/storage
- [ ] media not stored in git
- [ ] production storage is persistent (S3-like)
- [ ] private files not publicly accessible
- [ ] upload size/type validation exists
- [ ] download endpoints enforce authorization

---

# 33.12 Exercises (Do These Before Proceeding)

1. **Outbound API with retries**
   - Add a service that calls a fake external endpoint.
   - Mock it in tests (no real network).
   - Prove timeout is used and exceptions are mapped.

2. **Webhook replay protection**
   - Add timestamp tolerance test:
     - timestamp too old → 400 invalid_signature or timestamp error
   - Add event-id dedupe test (already shown; expand it with different provider).

3. **Webhook processing idempotency**
   - Add a unique “processed marker” in your domain (e.g., TaskEvent unique key from webhook id).
   - Ensure processing task can run twice without creating duplicate TaskEvent rows.

4. **Private download**
   - Make TaskExport downloads creator-only (or admin-only) and write tests for:
     - owner can download
     - non-member cannot access (404)
     - other member forbidden (403) depending on your policy

---

## 33.13 Chapter Summary

- External APIs must be called with timeouts, safe retries, and clear error mapping.
- Webhooks require signature verification + replay protection + idempotency + fast ACK.
- Background tasks are the right place for slow and retryable processing.
- File storage must be designed for persistence and privacy (public vs private media).
- The clean architecture pattern is:
  - views: thin
  - integrations/services: business mapping
  - tasks: retries + slow work
  - DB constraints: idempotency

---

Next chapter: **Part IX — Deployment and Production Operations (Chapter 36: Production Readiness Checklist)**  
We’ll translate everything you’ve built into a deployable, production-ready system:
settings hardening, static/media strategy, WSGI/ASGI choices, CI/CD, and runbooks.